In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
# variables

depts = ['77', '78', '91', '92', '93', '94']

criteres = {
    'budget_max': 230000,
    'surface_min': 60,
    'pieces': [3, 4],
    'distance_gare_max': 600,
    'distance_ecole_max': 250,
    'distance_espace_vert_max': 800
}

In [3]:
print("Chargement du fichier DVF géolocalisé...")

colonnes_dvf = [
    'date_mutation', 'nature_mutation', 'valeur_fonciere',
    'nom_commune', 'code_postal', 'code_departement',
    'type_local', 'surface_reelle_bati', 'nombre_pieces_principales',
    'longitude', 'latitude'
]

chunks = []

for chunk in pd.read_csv(
    'data/dvf.csv',
    usecols=colonnes_dvf,
    chunksize=500000,
    low_memory=False
):
    chunk['date_mutation'] = pd.to_datetime(chunk['date_mutation'], errors='coerce')
    chunk['annee'] = chunk['date_mutation'].dt.year
    
    # Filtrer uniquement 2022-2024
    chunk = chunk[chunk['annee'].isin([2022, 2023, 2024])]
    
    chunks.append(chunk)

df = pd.concat(chunks, ignore_index=True)
print(f"\nTotal chargé : {len(df):,} lignes")
print(f"\nRépartition par année :")
print(df['annee'].value_counts().sort_index())

Chargement du fichier DVF géolocalisé...

Total chargé : 11,937,532 lignes

Répartition par année :
annee
2022    4671911
2023    3806978
2024    3458643
Name: count, dtype: int64


In [4]:
print("Filtrage Île-de-France...")
df['code_departement'] = df['code_departement'].astype(str).str.zfill(2)
df = df[df['code_departement'].isin(depts)]
print(f"Après filtrage IDF : {len(df):,} lignes")

Filtrage Île-de-France...
Après filtrage IDF : 1,041,674 lignes


In [5]:
print("Filtrage Ventes & Appartements...")
df = df[df['nature_mutation'] == 'Vente']
df = df[df['type_local'] == 'Appartement']
print(f"Après filtrage Ventes/Appartements : {len(df):,} lignes")

Filtrage Ventes & Appartements...
Après filtrage Ventes/Appartements : 230,577 lignes


In [6]:
print("Filtrage 3-4 pièces & surface >= 60m²...")
df = df[df['nombre_pieces_principales'].isin(criteres['pieces'])]
df = df[df['surface_reelle_bati'] >= criteres['surface_min']]
print(f"Après filtrage pièces/surface : {len(df):,} lignes")

Filtrage 3-4 pièces & surface >= 60m²...
Après filtrage pièces/surface : 86,177 lignes


In [7]:
print("Nettoyage des valeurs manquantes...")
colonnes_critiques = ['valeur_fonciere', 'surface_reelle_bati', 'longitude', 'latitude']
df = df.dropna(subset=colonnes_critiques)

df['valeur_fonciere'] = df['valeur_fonciere'].astype(float)
df['surface_reelle_bati'] = df['surface_reelle_bati'].astype(float)

print(f"Après nettoyage NaN : {len(df):,} lignes")

Nettoyage des valeurs manquantes...
Après nettoyage NaN : 83,833 lignes


In [8]:
print("Calcul prix au m² et suppression outliers...")
df['prix_m2'] = df['valeur_fonciere'] / df['surface_reelle_bati']

outliers_min = 1
outliers_max = 5000
df = df[(df['prix_m2'] >= outliers_min) & (df['prix_m2'] <= outliers_max)]

print(f"Après suppression outliers ({outliers_min}-{outliers_max} €/m²) : {len(df):,} lignes")
print(f"\nStatistiques prix/m² :")
print(df['prix_m2'].describe())

Calcul prix au m² et suppression outliers...
Après suppression outliers (1-5000 €/m²) : 44,217 lignes

Statistiques prix/m² :
count    44217.000000
mean      3161.793472
std        994.551444
min          1.020408
25%       2446.808511
50%       3149.253731
75%       3925.373134
max       5000.000000
Name: prix_m2, dtype: float64


In [9]:
print("\nRépartition par année :")
print(df['annee'].value_counts().sort_index())

print("\nRépartition par département :")
print(df['code_departement'].value_counts().sort_index())

print("\nRépartition par nombre de pièces :")
print(df['nombre_pieces_principales'].value_counts())

print(f"\nShape final Section 2 : {df.shape}")
df.head()


Répartition par année :
annee
2022    17839
2023    13491
2024    12887
Name: count, dtype: int64

Répartition par département :
code_departement
77    7360
78    8995
91    8985
92    3957
93    7420
94    7500
Name: count, dtype: int64

Répartition par nombre de pièces :
nombre_pieces_principales
3.0    23707
4.0    20510
Name: count, dtype: int64

Shape final Section 2 : (44217, 13)


,date_mutation,nature_mutation,valeur_fonciere,code_postal,nom_commune,code_departement,type_local,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,annee,prix_m2
3554658,2022-01-07,Vente,173000.0,77550.0,Moissy-Cramayel,77,Appartement,61.0,3.0,2.588543,48.628815,2022,2836.065574
3554782,2022-01-10,Vente,170000.0,77176.0,Savigny-le-Temple,77,Appartement,60.0,3.0,2.571082,48.602748,2022,2833.333333
3554819,2022-01-07,Vente,139500.0,77210.0,Avon,77,Appartement,60.0,3.0,2.738240,48.410797,2022,2325.000000
3554834,2022-01-11,Vente,199000.0,77170.0,Brie-Comte-Robert,77,Appartement,60.0,3.0,2.611839,48.682449,2022,3316.666667
3554854,2022-01-06,Vente,135000.0,77350.0,Le Mée-sur-Seine,77,Appartement,84.0,4.0,2.641035,48.544868,2022,1607.142857


In [10]:
print("Chargement des gares RER/Métro/Train...")

gares = pd.read_csv('data/arrets.csv', sep=';')
print(f"Total arrêts chargés : {len(gares):,}")

print(f"\nTypes d'arrêts disponibles :")
print(gares['ArRType'].value_counts())

Chargement des gares RER/Métro/Train...
Total arrêts chargés : 38,469

Types d'arrêts disponibles :
ArRType
bus         35056
rail         2028
metro         811
tram          564
cableway       10
Name: count, dtype: int64


In [11]:
print("\nFiltrage metro, RER et tramway")

types_transport = ['metro', 'rail', 'tram']
gares = gares[gares['ArRType'].isin(types_transport)]

print(f"Après filtrage : {len(gares):,} gares/stations")
print(f"\nRépartition par type :")
print(gares['ArRType'].value_counts())


Filtrage metro, RER et tramway
Après filtrage : 3,403 gares/stations

Répartition par type :
ArRType
rail     2028
metro     811
tram      564
Name: count, dtype: int64


In [12]:
print("\nParsing des coordonnées GPS...")

def parse_geopoint(geopoint):
    if pd.isna(geopoint):
        return None, None
    try:
        lat, lon = geopoint.split(',')
        return float(lat.strip()), float(lon.strip())
    except:
        return None, None

gares[['latitude', 'longitude']] = gares['ArRGeopoint'].apply(
    lambda x: pd.Series(parse_geopoint(x))
)

gares = gares.dropna(subset=['latitude', 'longitude'])

print(f"Gares avec coordonnées valides : {len(gares):,}")
gares[['ArRName', 'ArRType', 'ArRTown', 'latitude', 'longitude']].head(10)


Parsing des coordonnées GPS...
Gares avec coordonnées valides : 3,403


,ArRName,ArRType,ArRTown,latitude,longitude
439,Bourg-la-Reine,rail,Bourg-la-Reine,48.780295,2.312172
1044,Saint-Cyr,tram,Versailles,48.799024,2.074185
1045,Fourqueux - Bel Air,tram,Saint-Germain-en-Laye,48.894921,2.069984
1046,Bailly,tram,Bailly,48.837205,2.074188
1047,Mareil-Marly,tram,Mareil-Marly,48.881043,2.079191
1048,Allée Royale,tram,Saint-Cyr-l'École,48.815447,2.078701
1049,Camp des Loges,tram,Saint-Germain-en-Laye,48.913933,2.081679
1060,Louveciennes,rail,Louveciennes,48.861253,2.122996
1069,Bécon les Bruyères,rail,Courbevoie,48.905318,2.268404
1108,Villiers-le-Bel - Gonesse - Arnouville,rail,Arnouville,48.993720,2.416548


In [13]:
print("\nConversion en GeoDataFrame...")

gares_gdf = gpd.GeoDataFrame(
    gares,
    geometry=gpd.points_from_xy(gares['longitude'], gares['latitude']),
    crs='EPSG:4326'
)

print(f"GeoDataFrame créé : {len(gares_gdf):,} gares")
print(f"CRS : {gares_gdf.crs}")
gares_gdf.head(3)


Conversion en GeoDataFrame...
GeoDataFrame créé : 3,403 gares
CRS : EPSG:4326


,ArRId,ArRVersion,ArRCreated,ArRChanged,ArRName,ArRType,ArRXEpsg2154,ArRYEpsg2154,ArRTown,ArRPostalRegion,ArRAccessibility,ArRAudibleSignals,ArRVisualSigns,ArRFareZone,ZdAId,ArRGeopoint,latitude,longitude,geometry
439,412842,1607889-1607888,2015-02-18T00:00:00+01:00,2024-09-07T16:30:39+02:00,Bourg-la-Reine,rail,649458,6853576,Bourg-la-Reine,92014,unknown,unknown,unknown,NaN,43097,"48.78029523889278, 2.312172431830233",48.780295,2.312172,POINT (2.31217 48.7803)
1044,481066,1274456-1555815,2021-05-31T00:00:00+02:00,2022-08-01T10:39:52+02:00,Saint-Cyr,tram,631996,6855837,Versailles,78646,true,unknown,unknown,4,480909,"48.79902437059493, 2.074185200862928",48.799024,2.074185,POINT (2.07419 48.79902)
1045,481070,1274472-1555829,2021-05-31T00:00:00+02:00,2022-08-01T10:39:52+02:00,Fourqueux - Bel Air,tram,631813,6866503,Saint-Germain-en-Laye,78551,true,unknown,unknown,4,480941,"48.894920794000285, 2.069983869456417",48.894921,2.069984,POINT (2.06998 48.89492)


In [14]:
print("Chargement des écoles...")

ecoles = pd.read_csv('data/annuaire-de-leducation.csv', sep=';')
print(f"Total établissements chargés : {len(ecoles):,}")

print(f"\nTypes d'établissements :")
print(f"Maternelles : {ecoles['Ecole_maternelle'].sum()}")
print(f"Élémentaires : {ecoles['Ecole_elementaire'].sum()}")

Chargement des écoles...
Total établissements chargés : 10,300

Types d'établissements :
Maternelles : 4245.0
Élémentaires : 4171.0


In [15]:
print("\nFiltrage écoles maternelles et élémentaires uniquement...")

ecoles = ecoles[
    (ecoles['Ecole_maternelle'] == 1) | 
    (ecoles['Ecole_elementaire'] == 1)
]

print(f"✓ Après filtrage : {len(ecoles):,} écoles")


Filtrage écoles maternelles et élémentaires uniquement...
✓ Après filtrage : 6,902 écoles


In [16]:
print("\nNettoyage des coordonnées GPS...")

ecoles = ecoles.dropna(subset=['latitude', 'longitude'])
ecoles['latitude'] = pd.to_numeric(ecoles['latitude'], errors='coerce')
ecoles['longitude'] = pd.to_numeric(ecoles['longitude'], errors='coerce')
ecoles = ecoles.dropna(subset=['latitude', 'longitude'])

print(f"Écoles avec coordonnées valides : {len(ecoles):,}")
ecoles[['Nom_etablissement', 'Nom_commune', 'latitude', 'longitude']].head(10)


Nettoyage des coordonnées GPS...
Écoles avec coordonnées valides : 6,891


,Nom_etablissement,Nom_commune,latitude,longitude
0,Ecole primaire Frida Kahlo,Chevilly-Larue,48.766282,2.367398
1,Ecole primaire Joséphine Baker,Vitry-sur-Seine,48.789495,2.395710
2,Ecole maternelle privée Forest international s...,Mareil-Marly,48.874391,2.064767
3,Ecole primaire privée Crayons de Miel,Boussy-Saint-Antoine,48.692434,2.535167
4,Ecole primaire publique La Fontaine,Issy-les-Moulineaux,48.822074,2.277927
5,Ecole élémentaire Jean Macé,Argenteuil,48.947610,2.250632
6,Ecole primaire privée de La Neuville,Chalmaison,48.497951,3.249128
7,Ecole maternelle privée Montessori,Avon,48.414360,2.731422
8,Ecole La Vallée des Hoyas,Mortcerf,48.788709,2.915969
9,Ecole primaire Marcel Cachin,Morsang-sur-Orge,48.653571,2.358427


In [17]:
print("\nConversion en GeoDataFrame...")

ecoles_gdf = gpd.GeoDataFrame(
    ecoles,
    geometry=gpd.points_from_xy(ecoles['longitude'], ecoles['latitude']),
    crs='EPSG:4326'
)

print(f"GeoDataFrame créé : {len(ecoles_gdf):,} écoles")
print(f"CRS : {ecoles_gdf.crs}")
ecoles_gdf.head(3)


Conversion en GeoDataFrame...
GeoDataFrame créé : 6,891 écoles
CRS : EPSG:4326


,Identifiant_de_l_etablissement,Nom_etablissement,Type_etablissement,Statut_public_prive,Adresse_1,Adresse_2,Adresse_3,Code_postal,Code_commune,Nom_commune,...,Code_type_contrat_prive,PIAL,etablissement_mere,type_rattachement_etablissement_mere,code_circonscription,code_zone_animation_pedagogique,libelle_zone_animation_pedagogique,code_bassin_formation,libelle_bassin_formation,geometry
0,0942569G,Ecole primaire Frida Kahlo,Ecole,Public,6 promenade Arthur Rimbaud,NaN,94550 CHEVILLY LARUE,94550,94021,Chevilly-Larue,...,99,NaN,NaN,NaN,0940936G,NaN,NaN,NaN,NaN,POINT (2.3674 48.76628)
1,0942617J,Ecole primaire Joséphine Baker,Ecole,Public,12 rue de Choisy,NaN,94400 VITRY SUR SEINE,94400,94081,Vitry-sur-Seine,...,99,NaN,NaN,NaN,0941461C,NaN,NaN,NaN,NaN,POINT (2.39571 48.78949)
2,0783768X,Ecole maternelle privée Forest international s...,Ecole,Privé,28 rue du Tour d'Echelle,NaN,78750 MAREIL MARLY,78750,78367,Mareil-Marly,...,10,NaN,NaN,NaN,0783228K,NaN,NaN,NaN,NaN,POINT (2.06477 48.87439)


In [18]:
print("Chargement des espaces verts...")

espaces_verts = pd.read_csv(
    'data/espaces-verts-et-boises-surfaciques-ouverts-ou-en-projets-douverture-au-public.csv', 
    sep=';'
)
print(f"Total espaces verts chargés : {len(espaces_verts):,}")

print(f"\nStatuts d'ouverture :")
print(espaces_verts['statouvlib'].value_counts())

Chargement des espaces verts...
Total espaces verts chargés : 12,978

Statuts d'ouverture :
statouvlib
Ouvert                  9780
Ouverture restreinte    1503
Projet d'ouverture      1344
Fermé                    294
Contrat d'ouverture       57
Name: count, dtype: int64


In [19]:
print("\nFiltrage espaces ouverts au public...")

espaces_verts = espaces_verts[espaces_verts['statutouv'] == 1]

print(f"Après filtrage : {len(espaces_verts):,} espaces verts ouverts")
print(f"\nCatégories d'espaces verts :")
print(espaces_verts['categlib'].value_counts().head(10))


Filtrage espaces ouverts au public...
Après filtrage : 9,780 espaces verts ouverts

Catégories d'espaces verts :
categlib
Square et jardin public                   4219
Espace boisé                              2769
Parc de ville                              737
Plaine de jeux                             637
Parc d'étang                               312
Grand parc urbain                          263
Parc sportif                               169
Autre                                      159
Espace naturel à caractère pédagogique     128
Jardin partagé                              92
Name: count, dtype: int64


In [20]:
print("\nParsing des coordonnées GPS...")

def parse_geo_point(geo_point):
    if pd.isna(geo_point):
        return None, None
    try:
        lat, lon = geo_point.split(',')
        return float(lat.strip()), float(lon.strip())
    except:
        return None, None

espaces_verts[['latitude', 'longitude']] = espaces_verts['Geo Point'].apply(
    lambda x: pd.Series(parse_geo_point(x))
)

espaces_verts = espaces_verts.dropna(subset=['latitude', 'longitude'])

print(f"Espaces verts avec coordonnées valides : {len(espaces_verts):,}")
espaces_verts[['nom', 'categlib', 'nomcom', 'latitude', 'longitude']].head(10)


Parsing des coordonnées GPS...
Espaces verts avec coordonnées valides : 9,780


,nom,categlib,nomcom,latitude,longitude
0,Bois Barbeterie,Espace boisé,Signy-Signets,48.918279,3.062662
2,Esplanade de Château de Sucy,Square et jardin public,Sucy-en-Brie,48.771315,2.522183
3,Jardin Dominique Chavoix,Square et jardin public,Suresnes,48.878408,2.229817
4,Place de l'Eglise,Square et jardin public,Valence-en-Brie,48.443849,2.890978
5,La Grande Garenne,Square et jardin public,Varennes-sur-Seine,48.374293,2.938540
6,Bois de Lieue,Espace boisé,Vauréal,49.036018,2.023880
7,Place Poulinat,Square et jardin public,Verrières-le-Buisson,48.746692,2.261758
9,Maison des Italiens,Square et jardin public,Versailles,48.801577,2.146317
10,Plaine de jeux,Plaine de jeux,Villebon-sur-Yvette,48.699160,2.248221
11,Forêt de Crécy,Espace boisé,Villeneuve-le-Comte,48.821464,2.859175


In [21]:
print("\nConversion en GeoDataFrame...")

espaces_verts_gdf = gpd.GeoDataFrame(
    espaces_verts,
    geometry=gpd.points_from_xy(espaces_verts['longitude'], espaces_verts['latitude']),
    crs='EPSG:4326'
)

print(f"GeoDataFrame créé : {len(espaces_verts_gdf):,} espaces verts")
print(f"CRS : {espaces_verts_gdf.crs}")
espaces_verts_gdf.head(3)


Conversion en GeoDataFrame...
GeoDataFrame créé : 9,780 espaces verts
CRS : EPSG:4326


,Geo Point,Geo Shape,objectid,insee,numero,nom,surfdonnee,statutouv,statouvlib,entreepay,...,st_length(shape),bev2024,j_rem,entree,num_p,catggenlib,empllib,latitude,longitude,geometry
0,"48.91827944662127, 3.0626617655607355","{""coordinates"": [[[3.0623839679027465, 48.9157...",1000,77451,1883,Bois Barbeterie,12.6,1,Ouvert,0,...,1678.620839,1,0,9,P1883,Espaces boisés et naturels,Sol,48.918279,3.062662,POINT (3.06266 48.91828)
2,"48.77131531402459, 2.522183031788077","{""coordinates"": [[[2.5221177699925628, 48.7717...",1017,94071,4178,Esplanade de Château de Sucy,8276.0,1,Ouvert,0,...,432.971643,1,0,0,P4178,Espaces verts,Sol,48.771315,2.522183,POINT (2.52218 48.77132)
3,"48.878407806366695, 2.2298171854163282","{""coordinates"": [[[2.229855003480298, 48.87876...",1024,92073,5170,Jardin Dominique Chavoix,2700.0,1,Ouvert,0,...,216.070826,1,0,9,P5170,Espaces verts,Sol,48.878408,2.229817,POINT (2.22982 48.87841)


In [22]:
print("Conversion du DataFrame DVF en GeoDataFrame...")

df_gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['longitude'], df['latitude']),
    crs='EPSG:4326'
)

print(f"GeoDataFrame créé : {len(df_gdf):,} biens")
print(f"CRS : {df_gdf.crs}")

Conversion du DataFrame DVF en GeoDataFrame...
GeoDataFrame créé : 44,217 biens
CRS : EPSG:4326


In [23]:
print("\nProjection en Lambert 93 (EPSG:2154) pour calculs métriques...")

df_gdf_projected = df_gdf.to_crs('EPSG:2154')
gares_gdf_projected = gares_gdf.to_crs('EPSG:2154')
ecoles_gdf_projected = ecoles_gdf.to_crs('EPSG:2154')
espaces_verts_gdf_projected = espaces_verts_gdf.to_crs('EPSG:2154')

print("Toutes les couches projetées en Lambert 93")


Projection en Lambert 93 (EPSG:2154) pour calculs métriques...
Toutes les couches projetées en Lambert 93


In [24]:
print("\nCalcul des distances aux gares/stations...")

df_with_gares = df_gdf_projected.sjoin_nearest(
    gares_gdf_projected[['geometry', 'ArRName', 'ArRType']],
    how='left',
    distance_col='distance_gare_m'
)

df_with_gares = df_with_gares.rename(columns={
    'ArRName': 'gare_la_plus_proche',
    'ArRType': 'type_transport'
})

print(f"Distances aux gares calculées")
print(f"Distance moyenne : {df_with_gares['distance_gare_m'].mean():.0f} m")
print(f"Distance médiane : {df_with_gares['distance_gare_m'].median():.0f} m")


Calcul des distances aux gares/stations...
Distances aux gares calculées
Distance moyenne : 1020 m
Distance médiane : 770 m


In [25]:
print("\nCalcul des distances aux écoles...")

df_with_ecoles = df_with_gares.sjoin_nearest(
    ecoles_gdf_projected[['geometry', 'Nom_etablissement']],
    how='left',
    distance_col='distance_ecole_m',
    rsuffix='_ecole'
)

df_with_ecoles = df_with_ecoles.rename(columns={
    'Nom_etablissement': 'ecole_la_plus_proche'
})

print(f"Distances aux écoles calculées")
print(f"Distance moyenne : {df_with_ecoles['distance_ecole_m'].mean():.0f} m")
print(f"Distance médiane : {df_with_ecoles['distance_ecole_m'].median():.0f} m")


Calcul des distances aux écoles...
Distances aux écoles calculées
Distance moyenne : 261 m
Distance médiane : 228 m


In [26]:
print("\nCalcul des distances aux espaces verts...")

df_final = df_with_ecoles.sjoin_nearest(
    espaces_verts_gdf_projected[['geometry', 'nom']],
    how='left',
    distance_col='distance_espace_vert_m',
    rsuffix='_ev'
)

df_final = df_final.rename(columns={
    'nom': 'espace_vert_le_plus_proche'
})

print(f"Distances aux espaces verts calculées")
print(f"Distance moyenne : {df_final['distance_espace_vert_m'].mean():.0f} m")
print(f"Distance médiane : {df_final['distance_espace_vert_m'].median():.0f} m")


Calcul des distances aux espaces verts...
Distances aux espaces verts calculées
Distance moyenne : 286 m
Distance médiane : 238 m


In [27]:
print("\nCréation des booléens pour les critères de John X...")

df_final['proche_transport'] = df_final['distance_gare_m'] <= criteres['distance_gare_max']
df_final['proche_ecole'] = df_final['distance_ecole_m'] <= criteres['distance_ecole_max']
df_final['proche_espace_vert'] = df_final['distance_espace_vert_m'] <= criteres['distance_espace_vert_max']
df_final['respecte_budget'] = df_final['valeur_fonciere'] <= criteres['budget_max']

print(f"Booléens créés")
print(f"\nRespect des critères :")
print(f"  - Proche transport (≤800m) : {df_final['proche_transport'].sum():,} biens ({df_final['proche_transport'].mean()*100:.1f}%)")
print(f"  - Proche école (≤800m) : {df_final['proche_ecole'].sum():,} biens ({df_final['proche_ecole'].mean()*100:.1f}%)")
print(f"  - Proche espace vert (≤1000m) : {df_final['proche_espace_vert'].sum():,} biens ({df_final['proche_espace_vert'].mean()*100:.1f}%)")
print(f"  - Dans budget (≤300k€) : {df_final['respecte_budget'].sum():,} biens ({df_final['respecte_budget'].mean()*100:.1f}%)")


Création des booléens pour les critères de John X...
Booléens créés

Respect des critères :
  - Proche transport (≤800m) : 21,003 biens (37.7%)
  - Proche école (≤800m) : 31,021 biens (55.7%)
  - Proche espace vert (≤1000m) : 54,536 biens (97.8%)
  - Dans budget (≤300k€) : 30,767 biens (55.2%)


In [28]:
print(f"Nombre de lignes actuel : {len(df_final):,}")
print(f"Nombre de lignes origine : {len(df):,}")

# Vérifier les duplicats sur les colonnes clés du bien immobilier
duplicates = df_final.duplicated(subset=[
    'date_mutation', 'valeur_fonciere', 'nom_commune', 
    'surface_reelle_bati', 'longitude', 'latitude'
])

print(f"\nNombre de duplicats : {duplicates.sum():,}")
print(f"Nombre de lignes uniques : {(~duplicates).sum():,}")

Nombre de lignes actuel : 55,740
Nombre de lignes origine : 44,217

Nombre de duplicats : 12,125
Nombre de lignes uniques : 43,615


In [29]:
print("Suppression des duplicats...")
print(f"Avant : {len(df_final):,} lignes")

df_final = df_final.drop_duplicates(subset=[
    'date_mutation', 'valeur_fonciere', 'nom_commune', 
    'surface_reelle_bati', 'longitude', 'latitude'
], keep='first')

print(f"Après : {len(df_final):,} lignes")
print(f"{57270 - len(df_final):,} duplicats supprimés")

Suppression des duplicats...
Avant : 55,740 lignes
Après : 43,615 lignes
13,655 duplicats supprimés


In [30]:
print(f"\nRespect des critères (après nettoyage) :")
print(f"  - Proche transport (≤800m) : {df_final['proche_transport'].sum():,} biens ({df_final['proche_transport'].mean()*100:.1f}%)")
print(f"  - Proche école (≤800m) : {df_final['proche_ecole'].sum():,} biens ({df_final['proche_ecole'].mean()*100:.1f}%)")
print(f"  - Proche espace vert (≤1000m) : {df_final['proche_espace_vert'].sum():,} biens ({df_final['proche_espace_vert'].mean()*100:.1f}%)")
print(f"  - Dans budget (≤300k€) : {df_final['respecte_budget'].sum():,} biens ({df_final['respecte_budget'].mean()*100:.1f}%)")


Respect des critères (après nettoyage) :
  - Proche transport (≤800m) : 16,122 biens (37.0%)
  - Proche école (≤800m) : 24,476 biens (56.1%)
  - Proche espace vert (≤1000m) : 42,668 biens (97.8%)
  - Dans budget (≤300k€) : 23,240 biens (53.3%)


In [31]:
print("\nCombination de TOUS les critères de John X...")

df_final['respecte_tous_criteres'] = (
    df_final['proche_transport'] &
    df_final['proche_ecole'] &
    df_final['proche_espace_vert'] &
    df_final['respecte_budget']
)

print(f"Biens respectant TOUS les critères : {df_final['respecte_tous_criteres'].sum():,} biens ({df_final['respecte_tous_criteres'].mean()*100:.1f}%)")

print(f"\nTop 10 communes avec le plus de biens parfaits :")
print(df_final[df_final['respecte_tous_criteres']]['nom_commune'].value_counts().head(10))


Combination de TOUS les critères de John X...
Biens respectant TOUS les critères : 4,533 biens (10.4%)

Top 10 communes avec le plus de biens parfaits :
nom_commune
Évry-Courcouronnes    195
Épinay-sur-Seine      180
Saint-Denis           153
Créteil               148
Savigny-sur-Orge      112
Corbeil-Essonnes      109
Noisy-le-Grand        106
Choisy-le-Roi         104
Rosny-sous-Bois        95
Brétigny-sur-Orge      91
Name: count, dtype: int64


In [32]:
print("Calcul du prix médian au m² par commune...")

prix_median_commune = df_final.groupby('nom_commune')['prix_m2'].median().reset_index()
prix_median_commune.columns = ['nom_commune', 'prix_m2_median_commune']

print(f"Prix médian calculé pour {len(prix_median_commune):,} communes")
print(f"\nTop 10 communes les plus chères (prix médian/m²) :")
print(prix_median_commune.sort_values('prix_m2_median_commune', ascending=False).head(10))

Calcul du prix médian au m² par commune...
Prix médian calculé pour 650 communes

Top 10 communes les plus chères (prix médian/m²) :
               nom_commune  prix_m2_median_commune
11   Arnouville-lès-Mantes             5000.000000
519              Saint-Yon             4960.317460
221             Grosrouvre             4852.631579
453            Porcheville             4812.500000
83      Bussy-Saint-Martin             4710.416667
567                    Ury             4705.882353
428               Orcemont             4666.666667
93               Chalifert             4662.790698
299             Le Vésinet             4661.538462
625               Viroflay             4619.746269


In [33]:
print("\nCalcul de l'évolution des prix 2022→2024 par commune...")

# Prix médian par commune et par année
prix_par_annee = df_final.groupby(['nom_commune', 'annee'])['prix_m2'].median().reset_index()

# Pivot pour avoir 2022, 2023, 2024 en colonnes
prix_pivot = prix_par_annee.pivot(index='nom_commune', columns='annee', values='prix_m2')

# Calcul évolution 2022→2024 (en %)
prix_pivot['evolution_prix_2022_2024_pct'] = (
    (prix_pivot[2024] - prix_pivot[2022]) / prix_pivot[2022] * 100
)

evolution_commune = prix_pivot[['evolution_prix_2022_2024_pct']].reset_index()
evolution_commune = evolution_commune.dropna()

print(f"Évolution calculée pour {len(evolution_commune):,} communes")
print(f"\nTop 10 communes avec la plus forte hausse :")
print(evolution_commune.sort_values('evolution_prix_2022_2024_pct', ascending=False).head(10))
print(f"\nTop 10 communes avec la plus forte baisse :")
print(evolution_commune.sort_values('evolution_prix_2022_2024_pct', ascending=True).head(10))


Calcul de l'évolution des prix 2022→2024 par commune...
Évolution calculée pour 454 communes

Top 10 communes avec la plus forte hausse :
annee         nom_commune  evolution_prix_2022_2024_pct
500           Saint-Mandé                    989.371981
117    Chenoise-Cucharmoy                    217.213664
245        Jouy-le-Châtel                    132.439336
236             Itteville                    117.427726
43            Bois-le-Roi                    109.866424
624             Vincennes                    101.092816
212                Gouaix                     99.811663
249               Juziers                     97.052237
161       Crouy-sur-Ourcq                     88.963826
156           Coutevroult                     78.192294

Top 10 communes avec la plus forte baisse :
annee              nom_commune  evolution_prix_2022_2024_pct
405                     Nangis                    -60.118508
36              Beton-Bazoches                    -57.103641
432         Ormes

In [ ]:
print("Filtrage des communes avec suffisamment de transactions...")

# Compter transactions par commune et année
nb_transactions = df_final.groupby(['nom_commune', 'annee']).size().reset_index(name='nb_transactions')
nb_pivot = nb_transactions.pivot(index='nom_commune', columns='annee', values='nb_transactions').fillna(0)

# Ne garder que les communes avec AU MOINS 10 transactions en 2022 ET 2024
communes_fiables = nb_pivot[
    (nb_pivot[2022] >= 10) & 
    (nb_pivot[2024] >= 10)
].index.tolist()

print(f"Communes avec stats fiables (≥10 transactions/an) : {len(communes_fiables)}")

# Filtrer les évolutions
evolution_commune_fiable = evolution_commune[
    evolution_commune['nom_commune'].isin(communes_fiables)
].copy()

print(f"\nTop 10 communes avec la plus forte hausse (fiable) :")
print(evolution_commune_fiable.sort_values('evolution_prix_2022_2024_pct', ascending=False).head(10))

print(f"\nTop 10 communes avec la plus forte baisse (fiable) :")
print(evolution_commune_fiable.sort_values('evolution_prix_2022_2024_pct', ascending=True).head(10))

Filtrage des communes avec suffisamment de transactions...
✓ Communes avec stats fiables (≥10 transactions/an) : 226

Top 10 communes avec la plus forte hausse (fiable) :
annee           nom_commune  evolution_prix_2022_2024_pct
415       Neuilly-sur-Seine                     77.999078
41            Bois-Colombes                     71.687332
235     Issy-les-Moulineaux                     67.906424
334       Magny-les-Hameaux                     16.608796
468             Romainville                     14.444035
509    Saint-Ouen-sur-Seine                     12.608503
135                  Clichy                     11.095038
365      Meulan-en-Yvelines                     10.610165
570                Valenton                      9.652568
609     Villennes-sur-Seine                      8.553682

Top 10 communes avec la plus forte baisse (fiable) :
annee           nom_commune  evolution_prix_2022_2024_pct
53                    Bondy                    -36.529129
60     Boulogne-Billa

In [35]:
print("\nJointure des agrégations avec le DataFrame principal...")

df_final = df_final.merge(prix_median_commune, on='nom_commune', how='left')
df_final = df_final.merge(evolution_commune_fiable, on='nom_commune', how='left')

print("✓ Agrégations ajoutées")


Jointure des agrégations avec le DataFrame principal...
✓ Agrégations ajoutées


In [36]:
print("\nCalcul de l'écart par rapport au prix médian de la commune...")

df_final['ecart_vs_median_pct'] = (
    (df_final['prix_m2'] - df_final['prix_m2_median_commune']) / 
    df_final['prix_m2_median_commune'] * 100
)

print("Écart vs médiane calculé")
print(f"\nStatistiques écart vs médiane :")
print(df_final['ecart_vs_median_pct'].describe())


Calcul de l'écart par rapport au prix médian de la commune...
Écart vs médiane calculé

Statistiques écart vs médiane :
count    43615.000000
mean        -0.009186
std         26.779870
min        -99.976237
25%        -13.580542
50%          0.000000
75%         13.384301
max        705.684217
Name: ecart_vs_median_pct, dtype: float64


In [ ]:
print("Nettoyage des colonnes avant export...")

# Supprimer les colonnes geometry et autres colonnes techniques inutiles
colonnes_a_garder = [
    'date_mutation', 'annee', 'valeur_fonciere', 'nom_commune', 
    'code_postal', 'code_departement', 'surface_reelle_bati', 
    'nombre_pieces_principales', 'longitude', 'latitude', 'prix_m2',
    'distance_gare_m', 'distance_ecole_m', 'distance_espace_vert_m',
    'gare_la_plus_proche', 'type_transport', 'ecole_la_plus_proche', 
    'espace_vert_le_plus_proche',
    'proche_transport', 'proche_ecole', 'proche_espace_vert', 'respecte_budget',
    'respecte_tous_criteres', 'prix_m2_median_commune', 
    'evolution_prix_2022_2024_pct', 'ecart_vs_median_pct'
]

# Créer DataFrame final propre (sans geometry)
df_export = df_final[colonnes_a_garder].copy()

print(f"DataFrame nettoyé : {df_export.shape}")
print(f"Colonnes finales : {len(df_export.columns)}")

Nettoyage des colonnes avant export...
✓ DataFrame nettoyé : (43615, 26)
Colonnes finales : 26


In [38]:
print("\nExport en format Parquet...")

df_export.to_parquet('data/data_final_idf.parquet', index=False)

print("✓ Fichier exporté : data/data_final_idf.parquet")
print(f"✓ Taille du fichier : {df_export.memory_usage(deep=True).sum() / 1024**2:.2f} MB en mémoire")


Export en format Parquet...
✓ Fichier exporté : data/data_final_idf.parquet
✓ Taille du fichier : 25.58 MB en mémoire


In [39]:
print("\n" + "="*60)
print("RAPPORT FINAL - PRÉPARATION DES DONNÉES")
print("="*60)

print(f"\nDonnées chargées :")
print(f"  - Période : 2022-2024")
print(f"  - Zone : Banlieue IDF (hors Paris)")
print(f"  - Total biens : {len(df_export):,}")

print(f"\nCouverture géographique :")
print(f"  - Départements : {df_export['code_departement'].nunique()}")
print(f"  - Communes : {df_export['nom_commune'].nunique()}")

print(f"\nCaractéristiques des biens :")
print(f"  - Type : Appartements uniquement")
print(f"  - Pièces : 3-4 pièces")
print(f"  - Surface min : {criteres['surface_min']} m²")
print(f"  - Prix moyen : {df_export['valeur_fonciere'].mean():,.0f} €")
print(f"  - Prix médian : {df_export['valeur_fonciere'].median():,.0f} €")
print(f"  - Prix/m² moyen : {df_export['prix_m2'].mean():,.0f} €/m²")

print(f"\nRespect des critères de John X :")
print(f"  - Proche transport (≤{criteres['distance_gare_max']}m) : {df_export['proche_transport'].sum():,} ({df_export['proche_transport'].mean()*100:.1f}%)")
print(f"  - Proche école (≤{criteres['distance_ecole_max']}m) : {df_export['proche_ecole'].sum():,} ({df_export['proche_ecole'].mean()*100:.1f}%)")
print(f"  - Proche espace vert (≤{criteres['distance_espace_vert_max']}m) : {df_export['proche_espace_vert'].sum():,} ({df_export['proche_espace_vert'].mean()*100:.1f}%)")
print(f"  - Dans budget (≤{criteres['budget_max']:,}€) : {df_export['respecte_budget'].sum():,} ({df_export['respecte_budget'].mean()*100:.1f}%)")
print(f"  - TOUS les critères : {df_export['respecte_tous_criteres'].sum():,} ({df_export['respecte_tous_criteres'].mean()*100:.1f}%)")

print(f"\nÉvolutions disponibles :")
print(f"  - Communes avec évolution fiable : {df_export['evolution_prix_2022_2024_pct'].notna().sum():,} biens")

print(f"\nTop 5 communes pour John X (biens parfaits) :")
top_communes = df_export[df_export['respecte_tous_criteres']]['nom_commune'].value_counts().head(5)
for i, (commune, count) in enumerate(top_communes.items(), 1):
    print(f"  {i}. {commune} : {count} biens")

print("\n" + "="*60)
print("PRÉPARATION TERMINÉE - Prêt pour le notebook applicatif !")
print("="*60)


RAPPORT FINAL - PRÉPARATION DES DONNÉES

Données chargées :
  - Période : 2022-2024
  - Zone : Banlieue IDF (hors Paris)
  - Total biens : 43,615

Couverture géographique :
  - Départements : 6
  - Communes : 650

Caractéristiques des biens :
  - Type : Appartements uniquement
  - Pièces : 3-4 pièces
  - Surface min : 60 m²
  - Prix moyen : 231,676 €
  - Prix médian : 225,000 €
  - Prix/m² moyen : 3,173 €/m²

Respect des critères de John X :
  - Proche transport (≤600m) : 16,122 (37.0%)
  - Proche école (≤250m) : 24,476 (56.1%)
  - Proche espace vert (≤800m) : 42,668 (97.8%)
  - Dans budget (≤230,000€) : 23,240 (53.3%)
  - TOUS les critères : 4,533 (10.4%)

Évolutions disponibles :
  - Communes avec évolution fiable : 39,907 biens

Top 5 communes pour John X (biens parfaits) :
  1. Évry-Courcouronnes : 195 biens
  2. Épinay-sur-Seine : 180 biens
  3. Saint-Denis : 153 biens
  4. Créteil : 148 biens
  5. Savigny-sur-Orge : 112 biens

PRÉPARATION TERMINÉE - Prêt pour le notebook applica